# Redis

Redis是远程字典服务器(Remote Dictionary Server)的缩写。

## 参考文献

* [Redis command reference](http://doc.redisfans.com/index.html)
* [Redis设计与实现](https://redisbook.readthedocs.io/en/latest/index.html)
* https://www.cnblogs.com/kevingrace/p/9004460.html

## 数据结构与对象

### 数据结构

Redis主要的数据结构

* 动态字符串
* 双端链表
* 字典
* 跳跃表
* 压缩列表
* 整数集合

跳跃表:

![](https://redisbook.readthedocs.io/en/latest/_images/skiplist.png)

### 字典

#### 解决健冲突

当有两个或以上数量的键被分配到了哈希表的同一个索引上面是，我们称这些键发生冲突(collision)。

Redis的哈希表使用链地址法来解决冲突，每个哈希表节点都有一个next指针，多个哈希表节点可以用next指针构成一个单向链表，被分配到同一个苏音上的多个节点可以用这个单向链表连接起来，这就解决了冲突链的问题。

#### rehash

随着操作的不断进行，哈希表保存的键值对会组建增多或减少，为了让哈希表的负载因子(load factor)维持一个合理的范围之内，当哈希表保存的键值对数量太多或太少时，程序需要对哈希表的大小进行呢相应的扩展或者收缩。

扩展和收缩哈希表的工作可以通过执行rehash(重新散列)操作来完成，Redis对字典的哈希表执行rehash的步骤如下：

1. 为字典的ht[1]哈希表分配空间，这个哈希表的空间大小取决于要执行的操作，以及ht[0]当前包含的键值对数量
    * 如果执行的是扩展操作，那么ht[1]的大小为第一个大于等于ht[0].used*2的$2^n$
    * 如果执行的是收缩操作，那么ht[1]的大小为第一个大于等于ht[0]。used*2的$2^n$
2. 将保存在ht[0]中的所有键值对rehash到ht[1]上面：rehash指的是重新计算健的哈希值和索引值，然后将键值对放置到ht[1]哈希表的制定位置上
3. 当ht[0]包含的所有键值对都迁移到了ht[1]之后，释放ht[0],将ht[1]设置为ht[0]，并在ht[1]新创建一个空白哈希表，未下一次rehash准备。

#### 渐进式rehash

### 对象

Redis 并没有直接使用上述的数据结构来实现键值对数据库，而是基于这些数据结构创建了一个对象系统。这个对象系统包括：

* String 字符串对象
* List 列表对象
* Hash 哈希对象
* Set 集合对象
* SortedSet 有序集合对象

**此外，Redis对象系统还实现了基于引用计数的内存回收机制，当程序不在使用某个对象的时候，这个对象占用的内存就会被自动释放；另外，还通过引用计数实现了对象共享机制，通过让多个数据库健共享同一个对象节约内存。**

### 列表

* *https://redisbook.readthedocs.io/en/latest/datatype/list.html#id4*
* *http://doc.redisfans.com/list/index.html*

BLPOP是列表的阻塞式(Blocking)弹出原语。

当给定列表内没有任何可弹出的时候，连接将被BLPOP命令阻塞直到等待超时或发现可弹出的元素为止。

**非阻塞行为**

当BLPOP被调用时，如果给定key内至少有一个非空列表，那么弹出遇到的第一个非空列表的头元素，并和被弹出元素所属的名字一起，组成结果返回给调用者。

**阻塞行为**

如果所有给定key都不存在或包含空列表，那么BLPOP命令将阻塞连接，直到等待超时，或另一个客户端对key的任意一个执行LPUSH或RPUSH命令为止。

**事件提醒**

有时候，为了等待一个新元素到达数据中，需要使用轮训的方式进行探查。

另一种更好的方式是，使用系统提供的阻塞原语，在新元素到达时立即进行处理，而新元素没有达到时，就一致阻塞，避免轮训占用资源。

### zset有序集合的实现原理

zset的结构同时包含一个字典和一个跳跃表。

```c
typeof struct zset {
    zskiplist *zsl;
    dict *dict;
} zset;
```

zset结构中zsl跳跃表按分值大小保存了所有集合元素，每个跳跃表节点都保存了一个结合元素。通过跳跃表，程序可以对有序集合进行范围型操作，比如ZRANK、ZRANGE等命令就是基于跳跃表来实现的。

**为什么有序集合需要同时使用跳跃表和字典来实现？**

**如果只使用字典来实现有序集合，那么虽然O(1)复杂度查找成员的分值这一特性会被保留，但是，因为字典以无序的方式来保存集合元素，所以每次在执行范围操作--比如ZRANK、ZRANGE等命令时，程序都需要对字典保存的所有元素进行排序，完成这种排序至少O(NlogN)时间复杂度，以及额外的O(N)内存。**

**如果只使用跳跃表来实现有序集合，那么跳跃表执行范围型操作的所有优点被保留，单因为没有了字典，所以根据成员查找分值这一操作的复杂度将从O(1)上升为O(logN)。因为以上原因，为了让有序集合的查找和范围型操作都尽可能快的执行，Redis选择了同时使用字典和跳跃表两种数据结构来实现有序集合。**

### 总结

如果只使用跳跃表来实现有序集合，那么跳跃表执行范围型操作的所有优点被保留，单因为没有了字典，所以根据成员查找分值这一操作的复杂度将从O(1)上升为O(logN)。因为以上原因，为了让有序集合的查找和范围型操作都尽可能快的执行，Redis选择了同时使用字典和跳跃表两种数据结构来实现有序集合。


* redis数据库中的每个键值对的健和值都是一个对象
* redis共有字符串、列表、哈希、集合、有序集合五种类型的对象
* 服务器在执行某些命令之前，会先检查给定健的类型能否执行指定的命令
* redis的对象系统带有引用计数实现的内存回收机制，当一个对象不再使用时，该对象所占用的内存就会被自动释放
* redis会共享0到9999的字符串对象
* 对象会记录自己的最后一次被访问的时间，这个时间可以用于计算对象的空转时间

## 数据库

### 切换数据库

```
SELECT 2
```

### 数据库健空间

健空间和用户所见的数据库是直接对应的：

* 健空间的健也就是数据库的健，每个键都是一个字符串对象
* 健空间的值也就是数据库的值，每个值可以是字符串对象、列表对象、哈希对象、集合对象和有序集合对象的任意一种Redis对象

### 过期健的删除策略

三种不同的删除策略：

* 定时删除， 设置键的过期时间的同时，创建一个定时器timer，让定时器在健的过期时间来临时，立即执行对健的删除操作
* 惰性删除，放任健过期不管，但是每次从健空间中获取健，都检查取得的键是否过期，如果过期的话，就删除键，如果没有过期，就返回键
* 定期删除，每隔一段时间，程序就对数据库进行一次检查，删除里面的过期键

**定时删除**

定时删除是对内存最友好的，缺点是对CPU时间不友好。

**惰性删除**

惰性删除对CPU时间是最友好的，缺点是对内存不友好。

**定期删除**

定期删除策是每隔一段时间执行一次删除过期键操作，并通过限制删除操作执行时长和频率来减少删除操作对CPU时间的影响。

### Redis的过期删除策略

**Redis服务器实际上使用的是惰性删除和定期删除两种策略：通过配合使用这两种删除策略，服务器可以很好地合理使用CPU时间和避免浪费内存空间之间取得平衡。**

#### 惰性删除策略的实现

所有读写数据库Redis命令之前，都会调用`expireIfNeeded`函数对输入键进行检查：

* 如果如何键已经过期，那么函数将输入键从数据库中删除
* 如果输入键未过期，那么函数不做动作

#### 定期删除策略的实现

过期键定期删除策略由`activeExpireCycle`函数实现，每当redis服务器周期性操作`serverCron`函数时，`activeExpireCycle`函数就会被调用，它在规定的时间，分多次遍历服务器中的各个数据库，从数据库的expires字典中检查一部分键的过期时间，并删除其中过期的键。

### AOF、RDB和复制功能对过期键的处理

#### 生成RDB文件

在执行SAVE命令或者BGSAVE命令创建一个新的RDB文件时，程序对数据库中键进行检查，已过期的键不会被保存到新创建的RDB文件中。

**数据库中包含过期键不会对新生成的RDB文件造成影响。**

#### 载入RDB文件

* 如果服务器以主服务器模式运行时，那么在载入RDB文件时，程序会对文件中保存的键进行检查，未过期的键会被载入数据库中，而过期的键会忽略，**所以过期键对载入RDB文件的主服务器不会造成影响**
* 如果服务器一从服务器模式运行，那么载入RDB文件时，文件中保存的所有键，不论过期与否，都会被载入到数据库中。不过，因为主服务器在进行数据同步到时候，从服务器的数据库就会被清空，所以，一般来讲，过期键对载入RDB文件到从服务器也不会造成影响

#### AOF文件写入

当服务器一AOF持久模式运行时，如果数据库中某个键已经过期，但它还没有被惰性删除或定期删除，那么AOF文件不会因为这个键过期而产生任何影响。

当过期键被惰性删除或定期删除后，程序会想AOF文件追加一条DEL命令，来显式记录该键已被删除。

#### 复制

当服务器运行在复制模式下时：

* 主服务器在删除一个过期键之后，会显式向所有从服务器发送一个DEL命令，告诉从服务器删除这个过期键
* 当从服务器在执行客户端发送读命令时，即使碰到过期健也不会将过期键删除，而是继续像处理未过期键一样来处理过期键
* 从服务器只有接到主服务器的DEL的命令时，才会删除过期键

### 总结

* 数据库主要由dict和expires两个字典构成，dict字典保存键值对，expire字典包含键的过期时间
* 数据库的键总是一个字符串对象，值可以是任意中Redis对象类型
* expires字典的键指向数据库中的某个键，而值记录了数据库键的过期时间，过期时间是一个以毫秒为单位的UNIX时间戳
* redis使用惰性删除和定期删除两种策略来删除过期健
* 执行SAVE或BGSAVE命令所产生的新的RDB文件不会包含过期的键
* 当主服务器删除一个过期键之后，它会向所有的从服务器发送一条DEL命令，显式删除过期键
* 从服务器收到主节点的DEL命令后删除该键，这种统一化、中心化的过期删除策略可以保证从服务器的数据一致性
* 当Redis命令对数据库修改之后，服务器会根据配置向客户端发送数据通知

## RDB持久化

因为Redis是内存数据库，它将数据库的状态存储在内存里，所以不想办法将存储在内存中的数据库状态保存在磁盘，那么一旦服务器进程退出，服务器中的数据库状态也会消失。

为了解决这个问题，redis提供了RDB持久化功能，这个功能将某个时间点上的数据库状态保存在RDB文件中。

RDB持久化既可以手动执行，也可以根据服务器配置选项定期执行，该功能可以将某个时间点上的数据库状态保存到一个RDB文件中。

### RDB文件的创建和载入

有两个命令可以用于生成RDB文件，一个是SAVE，另一个是BGSAVE。

**SAVE命令会阻塞Redis服务器进程，直到RDB文件创建完毕，在服务器阻塞期间，服务器不能处理任何命令请求。**

**和SAVE命令直接阻塞服务器进程的做法不同，`BGSAVE`命令会派生一个子进程，然后由子进程负责创建RDB文件，服务器进程继续处理命令请求。**

因为AOF文件更新频率通常比RDB文件的更新频率要高，所以：

* 如果服务器开启了AOF持久化功能，那么服务器会优先使用AOF文件还原数据库状态
* 只有AOF持久化处于关闭状态，服务器才会使用RDB文件还原数据库的状态

### SAVE命令执行时服务器的状态

SAVE命令执行时，Redis服务器会被阻塞，客户端发送的所有命令请求都会被阻塞。

### BGSAVE命令执行时的服务器状态

BGSAVE命令执行期间，客户端发送的SAVE命令会被服务器拒绝，服务器禁止SAVE命令和BGSAVE命令同时执行时为了避免父进程和子进程同时执行两个rdbSave调用，防止产生竞争条件。

## AOF持久化

除了RDB持久化功能，Redis还提供了AOF（Append Only File）持久化功能。与RDB持久化通过保存数据库的键值对来记录数据库的状态不同，AOF持久化是通过保存Redis服务器所执行的写命令来记录数据库的状态的。

### AOF重写

因为AOF持久化是通过保存被执行的写命令来记录数据库状态的，所以随着服务器运行时间的流逝，AOF文件中内容越来越多，文件的体积越来越大，如果不加以控制，体积过大的AOF文件很可能对服务器造成影响，并且AOF文件越大，使用AOF文件进行数据恢复所需要的时间就越长。

为了解决AOF文件体积膨胀问题，Redis提供了AOF文件重写功能。Redis服务器可以创建一个新的AOF文件替代现在的AOF文件，新旧两个AOF文件所保存的数据库状态相同，但新的AOF文件不包含任何浪费空间的冗余命令。

#### AOF重写的实现

AOF文件重写并不需要对现有的AOF文件进行读取、分析和写入操作，这个功能时通过读取服务器当前的数据库状态来实现的。

AOF重写是放在子进程里进行

* 子进程进行AOF重写，服务器进程可以继续处理命令请求
* 子进程带有服务器的数据副本，使用子进程而不是线程，可以避免使用锁的情况，保证数据的安全性

使用子进程也有一个问题，因为子进程在进行AOF重写期间，服务器进程还需要继续处理命令请求，而新的命令可能对现有的数据库状态做修改，从而使得服务器当前的数据库状态和重写后的AOF文件保存的数据库状态不一致。

为了解决这种数据不一致的问题，Redis服务器设置了一个AOF重写缓冲区，这个缓冲区在服务器创建子进程之后开始使用，当Redis服务器执行写完一个命令后，它同时将写命令发送给AOF缓冲区和AOF重写缓冲区。

### 总结

* AOF文件通过保存所有修改的数据库的写命令请求来记录服务器的状态
* AOF文件中的所有命令都以Redis命令请求协议的格式保存
* 命令请求会先保存到AOF缓冲区里面，之后定期写入并同步到AOF文件
* 服务器只要载入并重新执行AOF文件中的命令，就可以还原数据库本来的状态
* AOF重写是一个有歧义的名字，该功能是通过读取数据库的键值对来实现的，程序无需对AOF文件进行任何读入、分析和写入操作
* 执行`BGREWRIEAOF`命令时，Redis服务器会维护一个AOF重写缓冲区，该缓冲区在子进程创建AOF文件区间，记录服务器执行的所有写命令。当子进程完成创建新AOF文件后，服务器会将重写缓冲区的所有内容追加到AOF文件末尾，是的新旧两个AOF文件所保存的数据库状态一致。最后，服务器再用新的AOF文件替换就的AOF文件，以此完成AOF文件重写

## 发布与订阅

* 服务器状态在pubsub_channels字典保存了所有频道的订阅关系：SUBSCRIBE命令复杂将客户端和被订阅的平道关联到这个字典里面，UNSUBSCRIBE命令复杂解除客户端和被退订频道之间的关系
* 服务器状态在pubsub_patterns链表保存了所有模式的订阅关系：PSUBSCRIBE命令负责将客户端和被订阅客户端的模式记录到这个链表中，而PUNSUBSCRIBE命令则负责移除客户端和被退订模式在链表的记录
* PUBLISH命令通过访问pubsub_channels字典来向频道的所有订阅者发送消息，通过访问pubsub_patterns链表来向所有匹配频道的模式的订阅者发送消息
* PUBSUB命令的三个子命令都是读取pubsub_channels字典和pubsub_patterns链表中的信息来实现的

## 事务

Redis通过MULTI、EXEC、WATCH 等命令实现事务transaction功能。事务提供了一种将多个命令打包，然后一次性、按顺序地之行多个命令地机制，并且在事务执行期间，服务器不会中断事务而该去执行其他客户端地命令请求，它会将事务中的所有命令都执行完毕，然后才去处理其他客户端地命令请求。

Redis事务与传统的关系型事务的最大区别在于，Redis不支持事务会滚机制rollback，即使事务队列中的某个命令在执行期间出了错误，整个事务也会继续执行下去，直到事务队列中的所有命令执行完毕。

> Redis作者在事务功能的文档解释说，不支持事务会滚是因为这种复杂的功能和Redis追求见到高效的主旨不相符。

* 事务提供了一种将多个命令打包，然后一次性、有序执行的机制
* 多个命令会将入队到事务队列中，然后按先进先出的顺序执行
* 事务在执行过程中不会被打断，当事务队列中所有的命令执行完毕后，事务才会结束
* 带有WATCH命令的事务将会客户端和被监视的键在数据库的watched_keys字典中关联，当键被修改时，程序回将 所有监视被修改的客户端REDIS_DIRTY_CAS标志打开

## 集群

Redis集群是Redis提供的分布式数据库方案，集群通过分片(sharding)来进行数据共享，并提供复制和故障转移。

### 节点

一个Redis集群通常由三个多节点(node)组成，在刚开始的时候，每个节点都是互相独立的，它们都处于一个只包括自己的集群中，要组建一个真正可工作的集群，我们必须将各个独立的节点连接起来，构成一个包含多个节点的集群。

连接各个节点的工作可以使用`CLUSTER MEET`命令完成。

**节点和单机服务器在数据库方面的一个区别就死，节点只能使用0号数据库，而单机Redis服务器则没有这一限制。**

### 槽指派

Redis集群通过分片的方式来保存数据库的键值对：集群的整个数据库被分为16384个槽位(slot)，数据库中的每个都属于16384个槽的其中一饿，集群中的每个节点可以处理0个或最多16384个槽。

### 在集群中执行命令

当客户端向节点发送与数据库有关的命令时，接受命令的节点会计算出命令要处理数据库键属于哪个槽，并检查这个槽是否指派给了自己。

* 如果键所在的槽正好就指派给了当前节点，那么节点直接执行这个命令
* 如果键所在的槽位没有指派给当前节点，那么节点会向客户端返回一个MOVED错误，指向(redirect)至正确的节点，并在此发送之前想要执行的命令

### 重新分片

redis集群的重新分片操作可以将任意数量已经指派给某个点(源节点)的槽改为指派给另一个节点(目标节点)，并且相关槽所属的键值对也会从源节点移动目标及节点。

**重新分片操作可以在线(online)进行，在重新分片的过程中，集群不需要下线，并且源节点和目标节点都可以继续处理命令。**

### 复制与故障转移

略

### 总结

* 节点通过握手来将其他节点添加到自己所处的集群中
* 集群中的16384个槽可以分别指派给集群中的各个节点，每个节点都会记录哪些槽位指派了自己，那些槽位指派给了其他节点
* 节点在接到一个命令请求时，会先检查这个命令请求要处理的键所在的槽位是否自己负责，如果不是的话，节点将向客户端返回一个MOVED错误，MOVED错误携带的信息可以指引客户端转向正在负责相关槽位的节点
* 对Redis集群的重新分片工作是由redis-trib负责执行的，重新分片的关键就是将属于某一个槽的所有键从一个节点转移到另一个节点
* 如果节点A正在迁移槽i至我节点B，那么当节点A没能在自己的数据库中找到命令指定的数据库键时，节点A会向客户端B返回一个ASK错误，指引客户端到节点B继续查找指定的数据库键
* MOVED错误表示槽的负责权已经从一个节点转移到另一个节点，ASk错误只是两个节点在迁移槽的过程中使用一种临时措施
* 集群里的从节点用于复制主节点，并在主节点下线时，代替主节点继续处理命令请求
* 集群中的节点通过发送和接收消息来通信，通常的消息类型包括MEET、PING、PONG、PUBLiSH、FALL五种

## 分布式锁

* *https://www.cnblogs.com/rgcLOVEyaya/p/RGC_LOVE_YAYA_1003days.html*
* *https://www.cnblogs.com/fixzd/p/9479970.html*
* *https://redis.io/topics/distlock*
* *https://github.com/SPSCommerce/redlock-py*

### 单机版

获取锁

```redis
SET key val NX PX ttl
```

> 主要设置过期时间TTL的设置，TTL设置过小，有可能操作还没执行完，锁就被释放了

释放锁

```c
    if redis.call("get",KEYS[1]) == ARGV[1] then
        return redis.call("del",KEYS[1])
    else
        return 0
```

> 一定进行值的比较，保证释放的自己的加的锁。例如：client1 获取到锁，过期时间很短，因为导致锁被释放了，client2拿到了锁，client1如果不进行值的比较，就可能把client2持有的锁给释放了。

### 分布式版本

为了防止单节点宕机导致服务不可用，使用redis集群，集群的节点数为N。

获取锁

尝试对集群的所有节点加锁，如果加锁成功的节点超过`N//2 + 1`，则获取锁成功，否则，获取锁失败，释放所有节点的锁，获取锁的方式与单机版相同。

释放锁

多所有的集群所有的节点释放锁。